## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Import and read the charity_data.csv
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
application_df.isna().sum()

EIN                       0
NAME                      0
APPLICATION_TYPE          0
AFFILIATION               0
CLASSIFICATION            0
USE_CASE                  0
ORGANIZATION              0
STATUS                    0
INCOME_AMT                0
SPECIAL_CONSIDERATIONS    0
ASK_AMT                   0
IS_SUCCESSFUL             0
dtype: int64

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(["EIN", "NAME"], axis = 1, inplace = True)

In [4]:
# Determine the number of unique values in each column.

unique_count_list = [len(application_df[column].unique()) for column in application_df.columns]
pd.Series(unique_count_list, application_df.columns)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# Use the variable name `application_types_to_replace`
application_types_to_replace = application_df["APPLICATION_TYPE"].value_counts()[application_df["APPLICATION_TYPE"].value_counts() < 528].index

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2600        1
C1570        1
C2500        1
C1236        1
C1732        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df["CLASSIFICATION"].value_counts()[application_df["CLASSIFICATION"].value_counts() > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C4100        6
C2400        6
C1720        6
C1600        5
C1257        5
C1260        3
C2710        3
C0           3
C1246        2
C1256        2
C1267        2
C3200        2
C1234        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# Use the variable name `classifications_to_replace`
classifications_to_replace = application_df["CLASSIFICATION"].value_counts()[application_df["CLASSIFICATION"].value_counts() < 1883].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies = pd.get_dummies(application_df, drop_first = True)

In [11]:
# Split our preprocessed data into our features and target arrays
target = application_df_dummies["IS_SUCCESSFUL"]
features = application_df_dummies.drop("IS_SUCCESSFUL", axis = 1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state = 0)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
print(X_train_scaled.shape)
print(X_test_scaled.shape)

(25724, 36)
(8575, 36)


## Optimize the Model

##### First Attempt at Opitimization - add second hidden layer and changed optimizer

In [19]:
# Keras Optimizers

sgd = keras.optimizers.SGD(learning_rate = 0.01)

In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = keras.Sequential()
nn.add(keras.Input(shape = (X_train_scaled.shape[1],)))

# First hidden layer
nn.add(layers.Dense(64, activation = "relu"))

# Second hidden layer
nn.add(layers.Dense(64, activation = "relu"))

# Output layer
nn.add(layers.Dense(2))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 64)                2368      
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 2)                 130       
                                                                 
Total params: 6,658
Trainable params: 6,658
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Compile the model
nn.compile(loss = "binary_crossentropy", metrics = ["accuracy"], optimizer = sgd)

In [22]:
# Train the model
mc = keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', 
                                     save_weights_only=True, save_freq=5)
nn.fit(x = X_train, y = y_train, epochs = 20, callbacks = [mc])
       # validation_data = (X_test, y_test))

Epoch 1/20
804/804 [==============================] - 2s 2ms/step - loss: 7.0958 - accuracy: 0.5347: 0s - loss: 7.1076 - accu
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 7.0958 - accuracy: 0.5347
Epoch 3/20
804/804 [==============================] - 2s 2ms/step - loss: 7.0958 - accuracy: 0.5347
Epoch 4/20
804/804 [==============================] - 2s 2ms/step - loss: 7.0958 - accuracy: 0.5347
Epoch 5/20
804/804 [==============================] - 2s 2ms/step - loss: 7.0958 - accuracy: 0.5347
Epoch 6/20
804/804 [==============================] - 2s 2ms/step - loss: 7.0958 - accuracy: 0.5347
Epoch 7/20
804/804 [==============================] - 2s 2ms/step - loss: 7.0958 - accuracy: 0.5347
Epoch 8/20
804/804 [==============================] - 2s 2ms/step - loss: 7.0958 - accuracy: 0.5347
Epoch 9/20
804/804 [==============================] - 2s 2ms/step - loss: 7.0958 - accuracy: 0.5347
Epoch 10/20
804/804 [==============================] - 2s 2ms/step - loss:

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 4.1038 - accuracy: 0.5607 - 318ms/epoch - 1ms/step
Loss: 4.103777885437012, Accuracy: 0.5606997013092041


##### Second Attempt at Opitimization - add neurons to hidden layers and changing activation function

In [24]:
# Keras Optimizers

sgd = keras.optimizers.SGD(learning_rate = 0.01)

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = keras.Sequential()
nn.add(keras.Input(shape = (X_train_scaled.shape[1],)))

# First hidden layer
nn.add(layers.Dense(128, activation = "tanh"))

# Second hidden layer
nn.add(layers.Dense(128, activation = "tanh"))

# Output layer
nn.add(layers.Dense(2))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 128)               4736      
                                                                 
 dense_6 (Dense)             (None, 128)               16512     
                                                                 
 dense_7 (Dense)             (None, 2)                 258       
                                                                 
Total params: 21,506
Trainable params: 21,506
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compile the model
nn.compile(loss = "binary_crossentropy", metrics = ["accuracy"], optimizer = sgd)

In [27]:
# Train the model
mc = keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', 
                                     save_weights_only=True, save_freq=5)
nn.fit(x = X_train, y = y_train, epochs = 20, callbacks = [mc])
       # validation_data = (X_test, y_test))

Epoch 1/20
804/804 [==============================] - 2s 2ms/step - loss: 8.2474 - accuracy: 0.5347
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 8.2474 - accuracy: 0.5347
Epoch 3/20
804/804 [==============================] - 2s 3ms/step - loss: 8.2474 - accuracy: 0.5347
Epoch 4/20
804/804 [==============================] - 2s 3ms/step - loss: 8.2474 - accuracy: 0.5347
Epoch 5/20
804/804 [==============================] - 2s 2ms/step - loss: 8.2474 - accuracy: 0.5347
Epoch 6/20
804/804 [==============================] - 2s 3ms/step - loss: 8.2474 - accuracy: 0.5347
Epoch 7/20
804/804 [==============================] - 2s 3ms/step - loss: 8.2474 - accuracy: 0.5347
Epoch 8/20
804/804 [==============================] - 2s 3ms/step - loss: 8.2474 - accuracy: 0.5347
Epoch 9/20
804/804 [==============================] - 2s 3ms/step - loss: 8.2474 - accuracy: 0.5347
Epoch 10/20
804/804 [==============================] - 2s 3ms/step - loss: 8.2474 - accuracy: 0.5347

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 4.3190 - accuracy: 0.4687 - 245ms/epoch - 913us/step
Loss: 4.319024085998535, Accuracy: 0.468688040971756


##### Third Attempt at Opitimization - change optimizer to Adagrad and change activation back to relu

In [29]:
# Keras Optimizers

adg = keras.optimizers.Adagrad()

In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = keras.Sequential()
nn.add(keras.Input(shape = (X_train_scaled.shape[1],)))

# First hidden layer
nn.add(layers.Dense(128, activation = "relu"))

# Second hidden layer
nn.add(layers.Dense(128, activation = "relu"))

# Output layer
nn.add(layers.Dense(2))

# Check the structure of the model
nn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 128)               4736      
                                                                 
 dense_9 (Dense)             (None, 128)               16512     
                                                                 
 dense_10 (Dense)            (None, 2)                 258       
                                                                 
Total params: 21,506
Trainable params: 21,506
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Compile the model
nn.compile(loss = "binary_crossentropy", metrics = ["accuracy"], optimizer = adg)

In [32]:
# Train the model
mc = keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', 
                                     save_weights_only=True, save_freq=5)
nn.fit(x = X_train, y = y_train, epochs = 20, callbacks = [mc])
       # validation_data = (X_test, y_test))

Epoch 1/20
804/804 [==============================] - 2s 2ms/step - loss: 8.2474 - accuracy: 0.5347
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 8.2474 - accuracy: 0.5347
Epoch 3/20
804/804 [==============================] - 2s 3ms/step - loss: 8.2474 - accuracy: 0.5347
Epoch 4/20
804/804 [==============================] - 2s 3ms/step - loss: 8.2474 - accuracy: 0.5347
Epoch 5/20
804/804 [==============================] - 2s 2ms/step - loss: 8.2474 - accuracy: 0.5347
Epoch 6/20
804/804 [==============================] - 2s 3ms/step - loss: 8.2474 - accuracy: 0.5347
Epoch 7/20
804/804 [==============================] - 2s 3ms/step - loss: 8.2474 - accuracy: 0.5347
Epoch 8/20
804/804 [==============================] - 2s 3ms/step - loss: 8.2474 - accuracy: 0.5347
Epoch 9/20
804/804 [==============================] - 2s 2ms/step - loss: 8.2474 - accuracy: 0.5347
Epoch 10/20
804/804 [==============================] - 2s 3ms/step - loss: 8.2474 - accuracy: 0.5347

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 6.3997 - accuracy: 0.5699 - 217ms/epoch - 808us/step
Loss: 6.399679183959961, Accuracy: 0.5699125528335571
